In [1]:
import pandas as pd
import numpy as np
import requests
import pyodbc
import arcpy
from arcgis.features import FeatureLayer
# This is using Andy's Census API KEy
census_api_key = '9a73d08c296b844e58f1c70bd19c831826da5cbf'

# Need to define datatypes so that FIPS code doesn't get cast as int and drop leading 0s
dtypes = {
    'YEAR' : str,
    'STATE': str,
    'GEOGRAPHY': str,
    'GEOID': str,
    'TRPAID':str,
    'NEIGHBORHOOD': str
}

#Manually defined list of census tracts that are within the basin
 
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/27'

feature_layer = FeatureLayer(service_url)
tahoe_geometry_fields = ['YEAR', 'STATE', 'GEOGRAPHY', 'GEOID', 'TRPAID', 'NEIGHBORHOOD']
query_result = feature_layer.query(out_fields=",".join(tahoe_geometry_fields))
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
tahoe_geometry = pd.DataFrame([feature.attributes for feature in feature_list])

In [2]:
def get_jobs_data(year, census_geom_year, variable, variablename, census_api_key, tahoe_geometry, variable_category):
    base_url = 'https://api.census.gov/data/'
    df_total=pd.DataFrame()
    #Formatting to match html get request
    #get the zipcodes for inclusion from tahoe_geometry
    zipcodes = tahoe_geometry['geoid'].loc[(tahoe_geometry['census_geom_year']==census_geom_year)&(tahoe_geometry['GEOGRAPHY']=='ZIP CODE')]

    
    for zipcode in zipcodes:
        #print(f'{base_url}/{year}/cbp?get={variable}&for={geometry_return}:*&in=state:{state}%20county:{county}{geometry_level}&key={census_api_key}')
        request_url = f'{base_url}/{year}/cbp?get={variable}&for=zip%20code:{zipcode}&key={census_api_key}'
        response = requests.get(request_url)
        
        df = pd.DataFrame(response.json())
        #The json returns column names in the first row
        df.columns = df.iloc[0]
        df = df[1:]
        #Might as well add counties and states at this stage
        if df_total.empty:
            df_total=df
        else:
            df_total=pd.concat([df_total, df])
    return df_total

In [3]:
test = get_jobs_data('2021', 2021, 'EMP', 'Employees', census_api_key, tahoe_geometry, 'test')

KeyError: 'geoid'